In [ ]:
#Script to produce Coleman model fitting plot


import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d

# ---- Load and Process TD-DFT Data ----
data = np.loadtxt(r"combined_spectrum.txt")
x_nm = data[:, 0]
f = data[:, 1]

x_total = np.linspace(1, 400, 10000)
y = np.zeros(len(x_total))
for i in range(len(f)):
    y += (1.3062974e8) * (f[i] / (1e7 / 6200)) * np.exp(
        -(((1 / x_total) - (1 / x_nm[i])) / (1.0 / 6200)) ** 2
    )

x_ev = 1239.84 / x_total
x_ev_corrected = x_ev + 0.1
x_total_corrected = 1239.84 / x_ev_corrected
x_wavenumber = 10**3 / x_total_corrected

y *= 3.82353216e-3  # Convert to Mb
y /= 1542  # Normalize by number of Si atoms

# Interpolation for model fitting and plotting
interp_func = interp1d(x_wavenumber, y, kind='linear')
x_fit = np.linspace(3.0, 8.0, 500)
y_fit = interp_func(x_fit)

# ---- Define 2 Drude + Polynomial Model ----
def drude(x, x0, gamma):
    return x**2 / ((x**2 - x0**2)**2 + x**2 * gamma**2)

def drude_poly_model(x, a1, x01, gamma1, a2, x02, gamma2, c0, c1, c2):
    return (
        a1 * drude(x, x01, gamma1) +
        a2 * drude(x, x02, gamma2) +
        c0 + c1 * x + c2 * x**2
    )

# Fit the model
p0 = [2, 4.4, 0.4, 1.5, 4.7, 0.6, 0, 0, 0]
bounds = ([0, 4.0, 0.1, 0, 4.5, 0.1, -np.inf, -np.inf, -np.inf],
          [10, 4.6, 1.0, 10, 5.0, 1.0, np.inf, np.inf, np.inf])
weights = np.ones_like(x_fit)
weights[(x_fit >= 4.08) & (x_fit <= 4.84)] = 10.0
popt, _ = curve_fit(drude_poly_model, x_fit, y_fit, p0=p0, bounds=bounds, sigma=1/weights)

# Evaluate fitted model
fit_y = drude_poly_model(x_fit, *popt)

# Evaluate Components
fit_y_poly = drude_poly_model(x_fit, *popt)
drude1_poly = popt[0] * drude(x_fit, popt[1], popt[2])
drude2_poly = popt[3] * drude(x_fit, popt[4], popt[5])
poly_bg = popt[6] + popt[7] * x_fit + popt[8] * x_fit**2

# Calculate Chi-squared
residuals = y_fit - fit_y
sigma = 1 / weights
chi_squared = np.sum((residuals / sigma) ** 2)
dof = len(x_fit) - len(popt)  # degrees of freedom
reduced_chi_squared = chi_squared / dof

print(f"Chi-squared: {chi_squared:.3f}")
print(f"Reduced Chi-squared: {reduced_chi_squared:.3f}")


# ---- Plot (if needed) ----
plt.figure(figsize=(8, 6))
plt.plot(x_fit, y_fit, label='Mean Nanopyroxene Spectrum (Hydrogenated)', color='green')
plt.plot(x_fit, fit_y, '--', label='2 Drude + Polynomial Fit', color='black')
plt.plot(x_fit, drude1_poly, label='Drude Component 1', color='blue', linestyle=':')
plt.plot(x_fit, drude2_poly, label='Drude Component 2', color='cyan', linestyle=':')
plt.plot(x_fit, poly_bg, label='Polynomial Background', color='orange', linestyle=':')
plt.xlabel(r'Wavenumber ($\mu m^{-1}$)', fontsize=12)
plt.ylabel(r'$\sigma$ / Si atom [Mb]', fontsize=12)
plt.title('Nanopyroxene Spectrum with Coleman model fitting')
plt.legend()
plt.grid(True)
#plt.savefig("2_drude_polynomial_comparison.png", dpi=300)
plt.tight_layout()

plt.show()


In [ ]:
#Script to extract Table 2, S1, S2, S3 Parameters

import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d

# Drude function
def drude(x, x0, gamma):
    return x**2 / ((x**2 - x0**2)**2 + x**2 * gamma**2)

# Model: 2 Drudes + Polynomial
def drude_poly_model(x, a1, x01, gamma1, a2, x02, gamma2, c0, c1, c2):
    return a1 * drude(x, x01, gamma1) + a2 * drude(x, x02, gamma2) + c0 + c1 * x + c2 * x**2

# Dataset paths, labels, and number of silicon atoms
datasets = [
    ("combined_spectrum.txt", 1542)
]

x_fit = np.linspace(3.0, 8.0, 500)
results = []

for filename, label, si_atoms in datasets:
    data = np.loadtxt(
        f"{filename}"
    )
    x_nm = data[:, 0]
    f = data[:, 1]

    x_total = np.linspace(1, 400, 10000)
    y = np.zeros_like(x_total)
    for i in range(len(f)):
        y += (1.3062974e8) * (f[i] / (1e7 / 6200)) * np.exp(
            -(((1 / x_total) - (1 / x_nm[i])) / (1.0 / 6200)) ** 2
        )

    x_ev = 1239.84 / x_total
    x_ev_corrected = x_ev + 0.1
    x_total_corrected = 1239.84 / x_ev_corrected
    x_wavenumber = 10**3 / x_total_corrected
    y *= 3.82353216e-3
    y /= si_atoms

    interp_func = interp1d(x_wavenumber, y, kind='linear')
    y_fit = interp_func(x_fit)

    p0 = [2, 4.4, 0.4, 1.5, 4.7, 0.6, 0, 0, 0]
    bounds = ([0, 4.0, 0.1, 0, 4.5, 0.1, -np.inf, -np.inf, -np.inf],
              [10, 4.6, 1.0, 10, 5.0, 1.0, np.inf, np.inf, np.inf])
    weights = np.ones_like(x_fit)
    weights[(x_fit >= 4.08) & (x_fit <= 4.82)] = 10.0

    popt, _ = curve_fit(drude_poly_model, x_fit, y_fit, p0=p0, bounds=bounds, sigma=1/weights)

    drude1 = popt[0] * drude(x_fit, popt[1], popt[2])
    drude2 = popt[3] * drude(x_fit, popt[4], popt[5])
    combined_drude = drude1 + drude2

    peak_val = np.max(combined_drude)
    half_max = peak_val / 2
    indices = np.where(combined_drude >= half_max)[0]
    x_low = x_fit[indices[0]]
    x_high = x_fit[indices[-1]]
    fwhm = x_high - x_low
    center = x_fit[np.argmax(combined_drude)]

    results.append({
        "Label": label,
        "Si Atoms": si_atoms,
        "a1": popt[0], "x01": popt[1], "gamma1": popt[2],
        "a2": popt[3], "x02": popt[4], "gamma2": popt[5],
        "c0": popt[6], "c1": popt[7], "c2": popt[8],
        "Bump Center": center,
        "FWHM": fwhm,
        "Strength": peak_val
    })

# Display the results
df = pd.DataFrame(results)
df.to_csv("selected_spectrum_parameters.csv", index=False)